In [5]:
import openai
import os
from ebooklib import epub
import base64
import os
import requests

openai.api_key = "ENTER KEY" # get it at https://platform.openai.com/
stability_api_key = "ENTER KEY" # get it at https://beta.dreamstudio.ai/


def generate_cover_prompt(subject_matter):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=[
            {"role": "system", "content": "You are a creative assistant that writes a spec for the cover art of a book, based on the book's subject matter."},
            {"role": "user", "content": f"Subject matter: {subject_matter}\n\n--\n\nDescribe the cover we should create, based on the subject matter. This should be two sentences long, maximum."}
        ]
    )
    return response['choices'][0]['message']['content']


def create_cover_image(subject_matter):
    subject_matter = str(generate_cover_prompt(subject_matter))

    engine_id = "stable-diffusion-xl-beta-v2-2-2"
    api_host = os.getenv('API_HOST', 'https://api.stability.ai')
    api_key = stability_api_key

    if api_key is None:
        raise Exception("Missing Stability API key.")

    response = requests.post(
        f"{api_host}/v1/generation/{engine_id}/text-to-image",
        headers={
            "Content-Type": "application/json",
            "Accept": "application/json",
            "Authorization": f"Bearer {api_key}"
        },
        json={
            "text_prompts": [
                {
                    "text": subject_matter
                }
            ],
            "cfg_scale": 7,
            "clip_guidance_preset": "FAST_BLUE",
            "height": 768,
            "width": 512,
            "samples": 1,
            "steps": 30,
        },
    )

    if response.status_code != 200:
        raise Exception("Non-200 response: " + str(response.text))

    data = response.json()

    for i, image in enumerate(data["artifacts"]):
        with open(f"cover.png", "wb") as f:  # replace this if running locally, to where you store the cover file
            f.write(base64.b64decode(image["base64"]))


def create_epub(title, author, chapters, cover_image_path='cover.png'):
    book = epub.EpubBook()

    # Set metadata
    book.set_identifier('id123456')
    book.set_title(title)
    book.set_language('en')
    book.add_author(author)

    # Add cover image
    with open(cover_image_path, 'rb') as cover_file:
        cover_image = cover_file.read()
    book.set_cover('cover.png', cover_image)

    # Create chapters and add them to the book
    epub_chapters = []
    for i, chapter_dict in enumerate(chapters):
        full_chapter_title = list(chapter_dict.keys())[0]
        chapter_content = list(chapter_dict.values())[0]
        if ' - ' in full_chapter_title:
            chapter_title = full_chapter_title.split(' - ')[1]
        else:
            chapter_title = full_chapter_title

        chapter_file_name = f'chapter_{i+1}.xhtml'
        epub_chapter = epub.EpubHtml(title=chapter_title, file_name=chapter_file_name, lang='en')

        # Add paragraph breaks
        formatted_content = ''.join(f'<p>{paragraph.strip()}</p>' for paragraph in chapter_content.split('\n') if paragraph.strip())

        # Add line break before chapter title
        formatted_content = f"<br/><h1>{chapter_title}</h1>{formatted_content}"

        epub_chapter.content = formatted_content
        book.add_item(epub_chapter)
        epub_chapters.append(epub_chapter)

    # Define Table of Contents
    book.toc = epub_chapters

    # Add default NCX and Nav files
    book.add_item(epub.EpubNcx())
    book.add_item(epub.EpubNav())

    # Define CSS style
    style = '''
    @namespace epub "http://www.idpf.org/2007/ops";
    body {
        font-family: Cambria, Liberation Serif, serif;
    }
    h1 {
        text-align: left;
        text-transform: uppercase;
        font-weight: 200;
    }
    p {
        text-align: justify;
        text-justify: inter-word;
    }
    '''

    # Add CSS file
    nav_css = epub.EpubItem(uid="style_nav", file_name="style/nav.css", media_type="text/css", content=style)
    book.add_item(nav_css)

    # Create spine
    book.spine = ['nav'] + epub_chapters

    # Save the EPUB file
    epub.write_epub(f'{title}.epub', book)



import openai
import random
import json
import ast


def print_step_costs(response, model):
    input = response['usage']['prompt_tokens']
    output = response['usage']['completion_tokens']

    if model == "gpt-4" or model == "gpt-3.5-turbo-16k":
        input_per_token = 0.00003
        output_per_token = 0.00006
    if model == "gpt-3.5-turbo-16k":
        input_per_token = 0.000003
        output_per_token = 0.000004
    if model == "gpt-3.5-turbo-16k" or model == "gpt-4-32k":
        input_per_token = 0.00006
        output_per_token = 0.00012
    if model == "gpt-3.5-turbo" or model == "gpt-3.5-turbo-0613":
        input_per_token = 0.0000015
        output_per_token = 0.000002

    input_cost = int(input) * input_per_token
    output_cost = int(output) * output_per_token

    total_cost = input_cost + output_cost
    print('step cost:', total_cost)


def generate_subject_matter_prompt(subject_matter):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=[
            {"role": "system", "content": "You are a M&A consulting expert. Your task is to generate engaging subject matter for an educational book on selling a private company."},
            {"role": "user", "content": f"Generate 10 subject matters for an educational book on selling a private company based on this prompt: {subject_matter}"}
        ]
    )

    print_step_costs(response, "gpt-3.5-turbo-16k")

    return response['choices'][0]['message']['content'].split('\n')


def select_best_subject_matter(subject_matters):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=[
            {"role": "system", "content": "You are an expert in M&A consulting. Your task is to select the best subject matter for an educational book on selling a private company."},
            {"role": "user", "content": f"Here are a number of possible subject matters for an educational book on selling a private company: {subject_matters}\n\n--\n\nNow, select the best subject matter for the book based on the provided options or come up with a new and improved subject matter that is informative, engaging, and practical."}
        ]
    )

    print_step_costs(response, "gpt-3.5-turbo-16k")

    return response['choices'][0]['message']['content']


def improve_subject_matter(subject_matter):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=[
            {"role": "system", "content": "You are an expert in M&A consulting. Your task is to improve and refine the subject matter for an educational book on selling a private company."},
            {"role": "user", "content": f"Improve this subject matter: {subject_matter}"}
        ]
    )

    print_step_costs(response, "gpt-3.5-turbo-16k")

    return response['choices'][0]['message']['content']


def get_title(subject_matter):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=[
            {"role": "system", "content": "You are an expert writer."},
            {"role": "user", "content": f"Here is the subject matter: {subject_matter}\n\nWhat is the title of this educational book? Just respond with the title, do nothing else."}
        ]
    )

    print_step_costs(response, "gpt-3.5-turbo-16k")

    return response['choices'][0]['message']['content']


def write_first_chapter(subject_matter, first_chapter_title, writing_style):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=[
            {"role": "system", "content": "You are an expert in M&A consulting. Your task is to write the first chapter of an educational book on selling a private company."},
            {"role": "user", "content": f"Here is the subject matter to follow: {subject_matter}\n\nWrite the first chapter of this educational book: `{first_chapter_title}`.\n\nMake it informative, engaging, and well-written.\n\nHere is a description of the writing style you should use: `{writing_style}`\n\nInclude only the chapter text. There is no need to rewrite the chapter name."}
        ]
    )

    print_step_costs(response, "gpt-3.5-turbo-16k")

    improved_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=[
            {"role": "system", "content": "You are an expert in M&A consulting. Your task is to take the rough initial draft of the first chapter of the educational book on selling a private company and rewrite it to be significantly better."},
            {"role": "user", "content": f"Here is the subject matter you asked your assistant to follow: {subject_matter}\n\nHere is the first chapter they wrote: {response['choices'][0]['message']['content']}\n\nNow, rewrite the first chapter of this educational book in a way that is far superior. It should still cover the same subject matter but be much more detailed, engaging, and informative. Here is a description of the writing style you should use: `{writing_style}`"}
        ]
    )

    print_step_costs(improved_response, "gpt-3.5-turbo-16k")

    return improved_response['choices'][0]['message']['content']


def write_chapter(previous_chapters, subject_matter, chapter_title):
    try:
        i = random.randint(1, 2242)
        # write_to_file(f'write_chapter_{i}', f"Subject matter: {subject_matter}, Previous Chapters: {previous_chapters}\n\n--\n\nWrite the next chapter of this educational book, following the subject matter and taking into account the previous chapters as context. Here is the plan for this chapter: {chapter_title}\n\nWrite it informatively. Include only the chapter text. There is no need to rewrite the chapter name.")
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            messages=[
                {"role": "system", "content": "You are an expert in M&A consulting. Your task is to write the next chapter of an educational book on selling a private company."},
                {"role": "user", "content": f"Subject matter: {subject_matter}, Previous Chapters: {previous_chapters}\n\n--\n\nWrite the next chapter of this educational book, following the subject matter and taking into account the previous chapters as context. Here is the plan for this chapter: {chapter_title}\n\nWrite it informatively. Include only the chapter text. There is no need to rewrite the chapter name."}
            ]
        )

        print_step_costs(response, "gpt-3.5-turbo-16k")

        return response['choices'][0]['message']['content']
    except:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            messages=[
                {"role": "system", "content": "You are an expert in M&A consulting. Your task is to write the next chapter of an educational book on selling a private company."},
                {"role": "user", "content": f"Subject matter: {subject_matter}, Previous Chapters: {previous_chapters}\n\n--\n\nWrite the next chapter of this educational book, following the subject matter and taking into account the previous chapters as context. Here is the plan for this chapter: {chapter_title}\n\nWrite it informatively. Include only the chapter text. There is no need to rewrite the chapter name."}
            ]
        )

        print_step_costs(response, "gpt-3.5-turbo-16k")

        return response['choices'][0]['message']['content']


def generate_topic(prompt, num_chapters):
    print("Generating topic with chapters and high-level details...")
    json_format = """[{"Chapter CHAPTER_NUMBER_HERE - CHAPTER_TITLE_GOES_HERE": "CHAPTER_OVERVIEW_AND_DETAILS_GOES_HERE"}, ...]"""
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=[
            {"role": "system", "content": "You are an expert in M&A consulting. Your task is to write a detailed topic, complete with chapters, for an educational book on selling a private company."},
            {"role": "user", "content": f'Write a detailed topic with {num_chapters} chapters and high-level details based on this subject matter: {prompt}.\n\nDo it in this list of dictionaries format {json_format}'}
        ]
    )

    print_step_costs(response, "gpt-3.5-turbo-16k")

    improved_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=[
            {"role": "system", "content": "You are an expert in M&A consulting. Your task is to take the rough initial draft of the topic for an educational book on selling a private company and rewrite it to be significantly better. Pay attention to fix the spacing between words and sentences to be grammatically correct"},
            {"role": "user", "content": f"Here is the draft topic they wrote: {response['choices'][0]['message']['content']}\n\nNow, rewrite the topic in a way that is far superior. It should have the same number of chapters but be much improved in as many ways as possible. Remember to do it in this list of dictionaries format {json_format}"}
        ]
    )

    print_step_costs(improved_response, "gpt-3.5-turbo-16k")

    return improved_response['choices'][0]['message']['content']


def write_to_file(prompt, content):

    # Create a directory for the prompts if it doesn't exist
    if not os.path.exists('prompts'):
        os.mkdir('prompts')

    # Replace invalid characters for filenames
    valid_filename = ''.join(c for c in prompt if c.isalnum() or c in (' ', '.', '_')).rstrip()
    file_path = f'prompts/{valid_filename}.txt'

    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(content)

    print(f'Output for prompt "{prompt}" has been written to {file_path}\n')


def write_educational_book(subject_matter, num_chapters, writing_style):
    subject_matters = generate_subject_matter_prompt(subject_matter)

    best_subject_matter = select_best_subject_matter(subject_matters)

    improved_subject_matter = improve_subject_matter(best_subject_matter)


    title = get_title(improved_subject_matter)

    topic = generate_topic(improved_subject_matter, num_chapters)
    chapter_titles = ast.literal_eval(topic)


    educational_book = f"Topic:\n{topic}\n\n"

    first_chapter = write_first_chapter(topic, chapter_titles[0], writing_style.strip())
    educational_book += f"Chapter 1:\n{first_chapter}\n"
    chapters = [first_chapter]

    for i in range(num_chapters - 1):
        print(f"Writing chapter {i+2}...")  # + 2 because the first chapter was already added
        chapter = write_chapter(educational_book, topic, chapter_titles[i+1])
        educational_book += f"Chapter {i+2}:\n{chapter}\n"
        chapters.append(chapter)

    return educational_book, title, chapters, chapter_titles


# Example usage:
prompt = "The process of selling a private company"
num_chapters = 15
writing_style = "Clear and easily understandable, similar to a guidebook. Highly informative and practical."
educational_book, title, chapters, chapter_titles = write_educational_book(prompt, num_chapters, writing_style)

# Replace chapter descriptions with body text in chapter_titles
for i, chapter in enumerate(chapters):
    chapter_number_and_title = list(chapter_titles[i].keys())[0]
    chapter_titles[i] = {chapter_number_and_title: chapter}

# Create the cover
create_cover_image(str(chapter_titles))

# Create the EPUB file
create_epub(title, 'AI', chapter_titles, 'cover.png')



step cost: 0.02454
step cost: 0.02964
step cost: 0.051660000000000005
step cost: 0.0261
Generating topic with chapters and high-level details...
step cost: 0.15912
step cost: 0.20898
step cost: 0.15774
step cost: 0.22314
Writing chapter 2...
step cost: 0.2853
Writing chapter 3...
step cost: 0.33744
Writing chapter 4...
step cost: 0.3906
Writing chapter 5...
step cost: 0.42852
Writing chapter 6...
step cost: 0.47376
Writing chapter 7...
step cost: 0.51534
Writing chapter 8...
step cost: 0.59346
Writing chapter 9...
step cost: 0.6396000000000001
Writing chapter 10...
step cost: 0.72108
Writing chapter 11...
step cost: 0.76002
Writing chapter 12...
step cost: 0.79962
Writing chapter 13...
step cost: 0.84162
Writing chapter 14...
step cost: 0.89496
Writing chapter 15...
step cost: 0.93246
